In [ ]:
import torch
print(torch.__version__)

# cu가 안뜨면 CPU버전이다.

2.3.1+cu121


In [3]:
import sys
print(sys.version)

3.10.12 (main, Nov  4 2025, 08:48:33) [GCC 11.4.0]


In [ ]:
# CUDA(GPU) 사용 가능 여부 확인 -->주석자체가 프롬프트다.
# 셀에 주석을 잘 걸어놓으면 좋은 코드가 나온다.
import torch
print(torch.cuda.is_available())

# GPU 이름 확인
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce GTX 1660 SUPER


In [19]:
# 3. GPU가 사용 가능한 경우에만, 관련된 상세 정보 출력
# 코드 기억해야함
if torch.cuda.is_available():
    print("사용가능한 GPU 수 :", torch.cuda.device_count()) # 현장가면 엄청 많음
    current_gpu_index = torch.cuda.current_device()
    print("현재 사용 중인 GPU 인덱스 :", current_gpu_index) # GPU가 여러개일땐 인덱스 변경 가능, 분산처리도 가능
    print('Current GPU Name:', torch.cuda.get_device_name(current_gpu_index))
    print('Current GPU Memory:', torch.cuda.get_device_properties(current_gpu_index).total_memory / 1e9, "GB")
    print(f'\CUDA version: {torch.version.cuda}')

else:
    print("GPU가 사용 불가능합니다.")

#GPU의 메모리 사용량 확인
#중요한건 :AI를 제어하는 개발자가 되야함, 그래야 좋은 개발자임,




사용가능한 GPU 수 : 1
현재 사용 중인 GPU 인덱스 : 0
Current GPU Name: NVIDIA GeForce GTX 1660 SUPER
Current GPU Memory: 6.441992192 GB
\CUDA version: 12.1


In [ ]:
#4.간단한 텐서(Tensor) 생성 및 출력 
import torch    
x=torch.tensor([1,2,3]) # x는 변수: tensor라는 opject를 가지는 변수이다.
                    
print(x)
print(f'Type of x: {x.type()}')
# print(f'Type of x: {x.dtype()}' --> 에러발생함
print(f'Type of x: {x.dtype}') 
print(f'Size of x: {x.size()}')

print(f'Device of x: {x.device}')

# 데이터가 일부는 cpu,일부는 gpu에 저장하면 학습할때 멈출수 있어서 위에 코드 필요함

tensor([1, 2, 3])
Type of x: torch.LongTensor
Type of x: torch.int64
Size of x: torch.Size([3])
Device of x: cpu
